In [5]:
using DrWatson
@quickactivate "MEngProject"
using MEngProject

┌ Info: Precompiling MEngProject [d0493a11-efc0-4c7c-9b66-d0bd5a04cc55]
└ @ Base loading.jl:1260


In [24]:
using CUDA, DifferentialEquations, PyPlot, NNlib,  ImageFiltering, Images, MEngProject, MEngProject.LamKernels, MEngProject.Laminart, MEngProject.Utils, BenchmarkTools, Test

In [124]:
img = convert(CuArray{Float32,2}, load(datadir("Iine_100_100_gs.png")));
p = LaminartGPU.kernels(img, Parameters.parameters);
p = LaminartGPU.add_I_u_p(img, p);

In [125]:
tspan = (0.0f0, 100.0f0)
u0 = reshape(CUDA.zeros(Float32, p.dim_i, p.dim_j*(5*p.K+2)), p.dim_i, p.dim_j, 5*p.K+2,1);

In [126]:
x_lgn = reshape(CuArray{Float32}(undef, p.dim_i, p.dim_j * p.K), p.dim_i, p.dim_j, p.K,1)
C = reshape(CuArray{Float32}(undef, p.dim_i, p.dim_j * p.K), p.dim_i, p.dim_j, p.K,1)
H_z = reshape(CuArray{Float32}(undef, p.dim_i, p.dim_j * p.K), p.dim_i, p.dim_j, p.K,1)


f = LaminartGPU.MyFunction(x_lgn, C, H_z)

(::MEngProject.LaminartGPU.MyFunction{CuArray{Float32,4,CuArray{Float32,2,Nothing}}}) (generic function with 1 method)

In [127]:
prob = ODEProblem(f, u0, tspan, p)

ODEProblem with uType CuArray{Float32,4,CuArray{Float32,2,Nothing}} and tType Float32. In-place: true
timespan: (0.0f0, 100.0f0)
u0: Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

...

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

In [128]:
sol = solve(prob)

TaskFailedException: TaskFailedException:
MethodError: no method matching gemm!(::Val{false}, ::Val{false}, ::Int64, ::Int64, ::Int64, ::Float32, ::CuPtr{Float32}, ::CuPtr{Float32}, ::Float32, ::CuPtr{Float32})
Closest candidates are:
  gemm!(::Val, ::Val, ::Int64, ::Int64, ::Int64, ::Float32, !Matched::Ptr{Float32}, !Matched::Ptr{Float32}, ::Float32, !Matched::Ptr{Float32}) at /mnt/storage_1/users/cullinanen/.julia/packages/NNlib/sSn9M/src/gemm.jl:35
  gemm!(::Val, ::Val, ::Int64, ::Int64, ::Int64, !Matched::Float64, !Matched::Ptr{Float64}, !Matched::Ptr{Float64}, !Matched::Float64, !Matched::Ptr{Float64}) at /mnt/storage_1/users/cullinanen/.julia/packages/NNlib/sSn9M/src/gemm.jl:35
  gemm!(::Val, ::Val, ::Int64, ::Int64, ::Int64, !Matched::Complex{Float64}, !Matched::Ptr{Complex{Float64}}, !Matched::Ptr{Complex{Float64}}, !Matched::Complex{Float64}, !Matched::Ptr{Complex{Float64}}) at /mnt/storage_1/users/cullinanen/.julia/packages/NNlib/sSn9M/src/gemm.jl:35
  ...
Stacktrace:
 [1] ma

In [56]:
x .= u0[:, :, 1:p.K,:]
NNlib.conv(x, p.k_x_lgn, pad=0,flipped=true)

TaskFailedException: TaskFailedException:
MethodError: no method matching gemm!(::Val{false}, ::Val{false}, ::Int64, ::Int64, ::Int64, ::Float32, ::Ptr{Float32}, ::CuPtr{Float32}, ::Float32, ::Ptr{Float32})
Closest candidates are:
  gemm!(::Val, ::Val, ::Int64, ::Int64, ::Int64, ::Float32, ::Ptr{Float32}, !Matched::Ptr{Float32}, ::Float32, ::Ptr{Float32}) at /mnt/storage_1/users/cullinanen/.julia/packages/NNlib/sSn9M/src/gemm.jl:35
  gemm!(::Val, ::Val, ::Int64, ::Int64, ::Int64, !Matched::Float64, !Matched::Ptr{Float64}, !Matched::Ptr{Float64}, !Matched::Float64, !Matched::Ptr{Float64}) at /mnt/storage_1/users/cullinanen/.julia/packages/NNlib/sSn9M/src/gemm.jl:35
  gemm!(::Val, ::Val, ::Int64, ::Int64, ::Int64, !Matched::Complex{Float64}, !Matched::Ptr{Complex{Float64}}, !Matched::Ptr{Complex{Float64}}, !Matched::Complex{Float64}, !Matched::Ptr{Complex{Float64}}) at /mnt/storage_1/users/cullinanen/.julia/packages/NNlib/sSn9M/src/gemm.jl:35
  ...
Stacktrace:
 [1] macro expansion at /mn

In [89]:
p.k_x_lgn

1×1×2×1 CuArray{Float32,4,Nothing}:
[:, :, 1, 1] =
 0.5

[:, :, 2, 1] =
 0.5

In [75]:
p.k_x_lgn

1×1×2×1 CuArray{Float32,4,Nothing}:
[:, :, 1, 1] =
 0.5

[:, :, 2, 1] =
 0.5

In [53]:
x_lgn .= 0 
C .= 0 
H_z .=0
u0 .=0;

In [91]:
x_lgn

100×100×2×1 CuArray{Float32,4,CuArray{Float32,2,Nothing}}:
[:, :, 1, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0 

In [77]:
u0;

In [92]:
x = u0[:, :, 1:p.K,:]
NNlib.conv(x, p.k_x_lgn, pad=0, flipped=true)

100×100×1×1 CuArray{Float32,4,Nothing}:
[:, :, 1, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0

In [ ]:
x = u0
x_lgn = NNlib.conv(x, p.k_x_lgn, pad=0,flipped=true)

In [101]:
struct MyFunction{T} <: Function
  x_lgn::T
  C::T
  H_z::T
end

In [ ]:
MyFunction()

In [117]:
u = u0
du = u0
@inbounds begin
    
        x = @view u[:, :, 1:p.K,:]
        y = @view u[:, :, p.K+1:2*p.K,:]
        m = @view u[:, :, 2*p.K+1:3*p.K,:]
        z = @view u[:, :, 3*p.K+1:4*p.K,:]
        s = @view u[:, :, 4*p.K+1:5*p.K,:]

        #    C = @view u[:, :, 5*p.K+1:6*p.K]
        #    H_z = @view u[:, :, 6*p.K+1:7*p.K]

        v_p = @view u[:, :, 5*p.K+1,:]
        v_m = @view u[:, :, 5*p.K+2,:]
        #    x_lgn = @view u[:, :, 7*p.K+3]

        dx = @view du[:, :, 1:p.K,:]
        dy = @view du[:, :, p.K+1:2*p.K,:]
        dm = @view du[:, :, 2*p.K+1:3*p.K,:]
        dz = @view du[:, :, 3*p.K+1:4*p.K,:]
        ds = @view du[:, :, 4*p.K+1:5*p.K,:]

        dv_p = @view du[:, :, 5*p.K+1,:]
        dv_m = @view du[:, :, 5*p.K+2,:]

#         x_lgn = @view ff.x_lgn[:,:,1,:]
        #         x_lgn = similar(v_p)
        #         C = similar(x)
        #         H_z = similar(x)
        # x_lgn = Array{eltype(u)}(undef, p.dim_i, p.dim_j)
        #         C = reshape(Array{eltype(u)}(undef, p.dim_i, p.dim_j*p.K),p.dim_i,p.dim_j, p.K)
        #         C = reshape(zeros(p.dim_i, p.dim_j*p.K),p.dim_i,p.dim_j, p.K)
        # C = copy(u[:, :, 1:p.K])
        # H_z = copy(u[:, :, 1:p.K])

#         LaminartGPU.fun_x_lgn!(x_lgn, x, p)
#         LaminartGPU.fun_v_C!(C, v_p, v_m, p)
#         LaminartGPU.fun_H_z!(H_z, z, p)

        LaminartGPU.fun_dv!(dv_p, v_p, p.r, x_lgn, p)
#         LaminartGPU.fun_dv!(dv_m, v_m, .-p.r, x_lgn, p)
#         LaminartGPU.fun_dx_v1!(dx, x, C, z, p.x_V2, p)
#         LaminartGPU.fun_dy!(dy, y, C, x, m, p)
#         LaminartGPU.fun_dm!(dm, m, x, p)
#         LaminartGPU.fun_dz!(dz, z, y, H_z, s, p)
#         LaminartGPU.fun_ds!(ds, s, H_z, p)

    end

DimensionMismatch: DimensionMismatch("Input channels must match! (2 vs. 1)")

In [113]:
dv = similar(x_lgn)
LaminartGPU.conv!(dv, x_lgn, p.k_gauss_1, p)

DimensionMismatch: DimensionMismatch("Input channels must match! (2 vs. 1)")

In [115]:
size(x_lgn)

(100, 100, 2, 1)

In [116]:


p.k_gauss_1

5×5×1×1 CuArray{Float32,4,Nothing}:
[:, :, 1, 1] =
 0.00296902  0.0133062  0.0219382  0.0133062  0.00296902
 0.0133062   0.0596343  0.0983203  0.0596343  0.0133062
 0.0219382   0.0983203  0.162103   0.0983203  0.0219382
 0.0133062   0.0596343  0.0983203  0.0596343  0.0133062
 0.00296902  0.0133062  0.0219382  0.0133062  0.00296902

In [118]:
conv(x_lgn, p.k_gauss_1)

DimensionMismatch: DimensionMismatch("Input channels must match! (2 vs. 1)")

In [119]:
depthwiseconv(x_lgn, p.k_gauss_1)

DimensionMismatch: DimensionMismatch("Input channels must match! (2 vs. 1)")

In [123]:
u0
@view u[:, :, 5*p.K+1:5*p.K+1,:]
# m = @view u[:, :, 2*p.K+1:3*p.K,:]

100×100×1×1 view(::CuArray{Float32,4,CuArray{Float32,2,Nothing}}, :, :, 11:11, :) with eltype Float32:
[:, :, 1, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0